# Problema 2: invertire `ImL'`

Ovvero qualche cosa che non sia `ImL' \ g` ma che miri allo stesso risultato

## Setup

### Algebra degli operatori

Carichiamo il set minimo di dati e istruzioni

In [1]:
using LinearAlgebra

struct Operator  # Linear Matrix Operators from Matrices to Matrices (and the operator adjoint)
    op
    adj
    sym
end

## Operators
ℒ(A::Matrix)  = Operator(X->A*X   , X->A'*X, "ℒ$(size(A))"  )   # left multiply by A (X → AX)
ℛ(A::Matrix)  = Operator(X->X*A   , X->X*A', "ℛ$(size(A))")     # right multiply by A (X → XA)
ℋ(A::Matrix)  = Operator(X->X.*A  , X->X.*A, "ℋ$(size(A))")    # Hadamard product (elementwise product)
ℐ()  =          Operator(X->X      ,    X->X,    "I")     # identity operator
𝒪()  =           Operator(X->zero(X) , X->zero(X),"𝒪")# zero operator

𝒪 (generic function with 1 method)

Dobbiamo anche fare *overloading* delle operazioni

In [2]:
import Base:  zero, show, adjoint, *, \, ∘, +, -
show(io::IO, M::Operator) = print(io, M.sym)  # pretty printing
zero(::Any) = 𝒪() # Let's make any undefined zero the 𝒪 operator

adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
adjoint(B::Bidiagonal) = Bidiagonal(adjoint.(B.dv),adjoint.(B.ev),(B.uplo == 'U') ? :L : :U) # lower to upper

-(A::Operator) = Operator(X->-A.op(X), X->-A.adj(X),"-"*A.sym)
-(::typeof(𝒪()), X::Matrix) = -X  # 𝒪() - X should be -X
\(ℐ::typeof(ℐ()), A::Matrix) = A    # left division with ℐ() does nothing
*(A::Operator, X::Matrix) = A.op(X)
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
# We need [A;B]*C to somehow magically be [AC;BC]
*(M::Adjoint{Operator, Matrix{Operator}},v::Array) = M .* [v]
+(A::Array,x::Number)=A.+x

+ (generic function with 192 methods)

### La rete neurale

Riusiamo le celle di codice già viste...

In [3]:
using OffsetArrays

# funzione di attivazione
h(x)  = tanh(x)
h′(x) = 1 - h(x)^2


# questa funzione, di fatto, effettua il forward pass
function neural_net(params,X₀;h=h,h′= h′)
    T = Matrix{Float64}
    N = length(params)
    X = OffsetArray(Vector{T}(undef,N+1),0:N)   
    Δ = Vector{T}(undef, N)
    X[0] = X₀
    W = first.(params)
    B = last.(params)
    
    for i=1:N         
          X[i] =  h.(W[i]*X[i-1] .+ B[i])
          Δ[i] =  h′.(W[i]*X[i-1] .+ B[i])        
    end 
    X,Δ
end

neural_net (generic function with 1 method)

Adesso possiamo costruire la rete

In [4]:
## Un po' di parametri che descrivono la rete...

n = [5,4,3,1]   # contiene [n₀...n_N], le dimensioni dei layer
k = 10          # batchsize
N = length(n)-1 # numero di layer (nascosti + output). Dev'essere positivo

init(sizes...) = 0.01randn(sizes...)    # utility function per inizializzare

init (generic function with 1 method)

In [5]:
## Un po' di parametri degni di questo nome

# Creiamo le matrici dei pesi e i vettori dei bias
Ws_and_bs =[ [init(n[i+1],n[i]) , init(n[i+1])]  for i=1:N] # The second part of the pair is a vector here

X₀ = init(n[1],k)         # batch di k pattern (aka i dati)
y  = init(n[end],k);      # y is what we will compare X_N against (aka l'etichetta)

𝓁(x,y)  = sum(abs2,x-y)/2   # loss (errore quadratico)
𝓁′(x,y) = x .- y;           # derivata della loss (w.r.t. output layer)

X, δ = neural_net(Ws_and_bs,X₀) # Inferenza (aka forward pass)
# X e δ hanno i valori per ogni layer (servono per fare il backward pass)

([[0.026182801402408657 0.00036426890437413033 … -0.0035587775355718195 -0.0008974129296500301; 0.01262616136554745 -0.006078875137199755 … 0.022835625929291094 -0.0029997043926023206; … ; 0.007562379297229297 0.006364335033775799 … 0.004568393359911453 0.0032606974805106264; 0.02529801575551133 -0.010636318946584076 … -0.008903187121541572 -0.01584268278930539], [-0.0015643031674965498 -0.0016047417808474987 … -0.0012576775519790079 -0.0014414993127395603; 0.017556585929944604 0.018524073746863587 … 0.01877005165920578 0.018894364582394046; 0.0038256763183329564 0.003789147912606134 … 0.003528514863194329 0.0038549169780131834; 0.00012553809381422333 0.0002680403766767303 … 0.0004690012821301579 7.826798125503709e-5], [-0.01857867553484999 -0.01858206648049547 … -0.01858821758239703 -0.018586593970067396; -0.0057598891636786755 -0.005769121356940893 … -0.005769789575918278 -0.005771085676709476; -0.0014862254611881641 -0.0014860938874703851 … -0.0014841912710717852 -0.0014872400160731

Adesso possiamo calcolare il gradiente della loss usando la nostra tecnica

In [6]:
## The diagonal matrix
#M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i])] for i=1:N]) # OLD!
M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i]) ∘ ℛ(ones(1,k))] for i=1:N])

#M = Bidiagonal([ℋ(δ[i]) ∘ ℛ(X[i-1]) for i=1:N], [ℋ(δ[i]) ∘ ℛ(ones(1,k)) for i=1:(N-1)], :U)
#M = [M [fill(𝒪, N-1); ℋ(δ[N]) ∘ ℛ(ones(1,k))]]


## The lower triangular matrix (I-L)
ImL = Bidiagonal([ℐ() for i in 1:N], -[ℋ(δ[i]) ∘ ℒ(Ws_and_bs[i][1]) for i=2:N] , :L)

## derivata della loss (rispetto all'output layer)
g = [ fill(𝒪(),N-1) ; [𝓁′(X[N],y)] ]      

## Finalmente, il gradiente della loss, usando il backslash
∇J = M' * (ImL' \ g)

3-element Vector{Matrix{Matrix{Float64}}}:
 [[1.3845378281510947e-9 -6.24730014455307e-9 … 2.499872265164586e-9 -5.191643822861122e-10; -3.298363595619627e-9 1.4883797988471203e-8 … -5.956030589801615e-9 1.2372346476169532e-9; -4.064940540923755e-9 1.834174463075297e-8 … -7.339501371341228e-9 1.5242371843428107e-9; 3.2975087298229425e-9 -1.48789746340383e-8 … 5.953852680296688e-9 -1.236472586295571e-9]; [-8.357913104048513e-7; 1.991220942999854e-6; 2.453841022163965e-6; -1.9905747616232742e-6;;];;]
 [[3.485723529350209e-8 -5.197243227015326e-7 -9.72551972410096e-8 -3.5491482104977278e-9; 2.104619283457053e-7 -3.1380051299746403e-6 -5.872099785997063e-7 -2.1429139209242083e-8; 7.32860724580466e-7 -1.0927015288229223e-5 -2.044755232443511e-6 -7.461955670948799e-8]; [-2.8164590140625356e-5; -0.00017005290032392258; -0.0005921502872866001;;];;]
 [[0.0010855019765815231 0.00033665014580328153 8.67505374656994e-5]; [-0.058399619525367034;;];;]

### 1. Invertire esplicitamente

Se invece di fare `M' * (ImL' \ g)` volessimo associare a sinistra? Ci servirebbe l'inversa di `ImL'`. C'è un modo molto naive per farlo...

In [7]:
try
    inv(ImL)
catch e 
    str = IOBuffer()
    showerror(str, e)
    msg = String(take!(str))
    msg = msg[1:findfirst('\n', msg)-1]
end

"MethodError: no method matching one(::Type{Operator})"

Ok, insegnamogli chi è l'identità moltiplicativa...

In [8]:
Base.one(::typeof(Operator)) = ℐ()

E già che ci siamo insegnamogli un altro po' di aritmetica

In [9]:
# Il prodotto tra operatori è la loro composizione
*(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)

# Somma tra operatori, definita nel modo ovvio
+(A::Operator, B::Operator) = Operator(X->(A.op(X) + B.op(X)), X->(A.adj(X) + B.adj(X)), A.sym*" + "*B.sym)

# Differenza tra operatori, definita nel modo ovvio
-(A::Operator, B::Operator) = A + (-B)

- (generic function with 199 methods)

In [10]:
try
    inv(ImL)
catch e 
    str = IOBuffer()
    showerror(str, e)
    msg = String(take!(str))
    msg = msg[1:findfirst('\n', msg)-1]
end

"MethodError: no method matching Operator(::Operator)"

Ci tocca dire al costruttore di tipo che un `Operator` è già un `Operator`...

In [11]:
Operator(op::Operator) = op

Operator

In [12]:
try
    inv(ImL)
catch e 
    str = IOBuffer()
    showerror(str, e)
    msg = String(take!(str))
    msg = msg[1:findfirst('\n', msg)-1]
end

"MethodError: no method matching inv(::Operator)"

Il problema è risolto, ma comunque non siamo in grado di invertire.

***OSS***: possiamo tirare tutte le martellate che ci pare. Prima o poi ci scontreremo con il fatto che `inv(Operator)` non è definito, e per buona ragione

#### Inversi di operatori? Operatori inversi?

A questo punto, ci scontriamo con un **limite invalicabile**: per fare quest'operazione bisogna **invertire operatori**, e non tutti gli operatori sono invertibili. Nello "spazio in cui stiamo operando" (quello degli omomorfismi tra spazi di matrici) mancano le "garanzie" che ci permetterebbero di "fare quel che abbiamo intenzione di fare".

Nonostante ciò, si potrebbe pensare ad un **workaround**: estendere lo `struct Operator` con altri due campi, a rappresentare l'omomorfismo inverso e l'aggiunto dell' inverso.

Incontreremmo però un altro **roadblock**: mentre la somma, la differenza, la moltiplicazione per scalare sono lineari, mentre la trasposizione si comporta bene, l'inversione è assai più ostica.

Ad esempio,
- $f + g$ può essere invertibile senza che né $f$ né $g$ lo siano
- Non vale $(f+g)^{-1}=f^{-1} + g^{-1}$. Se $f$ è invertibile, si potrebbe pensare di fare $(\sum_{k=0}^{\infty} (-f^{-1}\circ g)^k)\circ f^{-1}$
    - Se né $f$ né $g$ fossero invertibili?
    - A che punto troncare la serie?
    - Si tratta di un calcolo estremamente costoso, per una singola valutazione dell'omomorfismo somma...

### 2. Fattorizzazione LU  

Ok, invertire è stupido. 

Abbiamo detto che per valutare `ImL' \ g` si risolve un sistema lineare tramite sostituzione all'indietro. Nella presentazione abbiamo visto che le operazioni svolte da questa *backsubstitution* corrispondono a quelle fatte dall'algoritmo di *backpropagation*.

Sempre un sistema lineare resta. In generale, per matrici dense e non troppo grandi, li si risolve con fattorizzazione LU. Funzionerebbe?

In [13]:
try
    F = lu(ImL')
catch e
    str = IOBuffer()
    showerror(str, e)
    msg = String(take!(str))
    msg = msg[1:findfirst('\n', msg)-1]
end

"MethodError: no method matching /(::Operator, ::Operator)"

Ok. A quanto pare nella LU non c'è un check preliminare a controllare che la matrice da fattorizzare sia già triangolare superiore. Non è grave, l'importante è che il `\` ce lo abbia (dopodiché ci si affida all'intelligenza dell'utente...)

Però se i nodi non sono topologicamente ordinati, fare la LU è obbligatorio, perché la matrice di adiacenza non è in forma triangolare superiore.

In [14]:
# permutiamo le etichette 4 e 5, ovvero dei due hidden layer della rete
p = [2, 1, 3]
ImLp = ImL[p, p]
Mp = M[:, p]

display(ImLp)

3×3 Matrix{Operator}:
 I                  -ℋ(3, 10)∘ℒ(3, 4)  𝒪
 𝒪                  I                  𝒪
 -ℋ(1, 10)∘ℒ(1, 3)  𝒪                  I

In [18]:
try
    F = lu(ImLp')    
catch e
    str = IOBuffer()
    showerror(str, e)
    msg = String(take!(str))
    msg = msg[1:findfirst('\n', msg)-1]
end

"MethodError: no method matching /(::Operator, ::Operator)"